In [ ]:

# define filter length in seconds, convert to samples
filter_len = filter_length * run_parameters["update_rate"]

# Initialize roi_trfs - ROI x Filter x Trials
roi_trfs = np.zeros(
    (
        roi_data["epoch_response"].shape[0],
        int(filter_len),
        int(run_parameters["num_epochs"]),
    )
)


# iterate over ROIs to save STRFs and movies

for roi_id in range(0, roi_data["epoch_response"].shape[0]):


    # initialize strf by trial array (T,Tr)
    roi_trf = np.zeros((int(filter_len), int(run_parameters["num_epochs"])))
    for trial_num in range(0, int(run_parameters["num_epochs"])):
        current_resp = roi_data["epoch_response"][roi_id, trial_num]
        trimmed_current_resp = current_resp[int(pre_frames):int(stim_frames_end)]

        # initialize strf and full time series for update rate of stimulus
        full_t = np.arange(
            run_parameters["pre_time"],
            run_parameters["stim_time"] + run_parameters["pre_time"],
            1 / run_parameters["update_rate"],
        )
        trf = np.zeros(int(filter_len))
        # linearly interpolate response to match stimulus timing
        #full_resp = np.interp(full_t, roi_data["time_vector"], current_resp)
        resampled_current_resp=signal.resample(trimmed_current_resp,len(all_stims[:,-1]))


        resp_mean = np.mean(resampled_current_resp)
        resp_var = np.var(resampled_current_resp)
        # compute TRF for mean-subtracted stimulus and response
        n = all_stims.shape[1]
        ext_size = 2 * n - 1
        fsize = 2 ** np.ceil(np.log2(ext_size)).astype("int")
        vis_stim = all_stims[:, trial_num]
        vis_mean = np.mean(vis_stim)
        filter_fft = np.fft.fft(resampled_current_resp - resp_mean, fsize) * np.conj(
            np.fft.fft(vis_stim - vis_mean, fsize)
        )
        filt = np.real(np.fft.ifft(filter_fft))[0 : int(filter_len)]
        trf = np.flip(filt)
        roi_trf[:, trial_num] = trf

    roi_trfs[roi_id, :, :] = roi_trf


# SPLIT into NO Opto and YES Opto trials (alternating)
no_slice = roi_trfs[:, :, 0::2]  # Every 2 trials, starting at 0
yes_slice = roi_trfs[:, :, 1::2]  # Every 2 trials, starting at 1

# compute mean STRF
# roi_mean_strf = np.mean(roi_strf,3);
roi_mean_trf = np.mean(roi_trfs, 2)
nopto_mean_trf = np.mean(no_slice, 2)
yopto_mean_trf = np.mean(yes_slice, 2)

# Standard Error of the Mean calculations
nopto_sem = sem(no_slice, axis=2)  # calculate the no opto sem
nopto_sem_plus = np.squeeze(nopto_mean_trf + nopto_sem)
nopto_sem_minus = np.squeeze(nopto_mean_trf - nopto_sem)
yopto_sem = sem(yes_slice, axis=2)  # calculate the yes opto sem
yopto_sem_plus = np.squeeze(yopto_mean_trf + yopto_sem)
yopto_sem_minus = np.squeeze(yopto_mean_trf - yopto_sem)

# Checking ouputs for separating no opto from opto trials
print("\n---------------------------------------------------------------------")
print("---------------------------------------------------------------------")
print("Checking the shape of the various trfs to ensure opto/no opto split!")
print("Shape of roi_trfs is: " + str(roi_trfs.shape))
print("Shape of no_slice is: " + str(no_slice.shape))
print("Shape of yes_slice is: " + str(yes_slice.shape))

print("\nShape of roi_mean_trf is: " + str(roi_mean_trf.shape))
print("Shape of nopto_mean_trf is: " + str(nopto_mean_trf.shape))
print("Shape of yopto_mean_trf is: " + str(yopto_mean_trf.shape))